In [64]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import spacy

In [65]:
nlp = spacy.load("en_core_web_sm")

In [66]:
chats = {"intents": 
    [
    {
      "tag": "greeting",
      "patterns": 
      [
        "Hi",
        "Hey",
        "How are you",
        "Is anyone there?",
        "Hello",
        "Good day",
        "Good evening",
        "Good night"
      ],
      "responses": 
      [
        "Hey!",
        "Hello, thanks for visiting",
        "Hi there, what can I do for you?",
        "Hi there, how can I help?",
        "What's up?"
      ]
    },
    {
      "tag": "goodbye",
      "patterns": 
        [

        "Bye", 
        "See you later", 
        "Goodbye"
        
        ],

      "responses": 
      [
        "See you later, thanks for visiting",
        "Have a nice day",
        "Bye! Come back again soon."
      ]
    },
    {
      "tag": "thanks",
      "patterns": ["Thanks", "Thank you", "That's helpful", "Thank's a lot!"],
      "responses": ["Happy to help!", "Any time!", "My pleasure"]
    },
    {
      "tag": "Mood",
      "patterns": [
          "I don't feel well",
          "I feel exhausted",
          "I am tired"

      ],
      "responses": [
        "Do you have any symptoms",
        "Can you specify your symptoms"
      ]
    },
        {
      "tag": "Appointment",
      "patterns": [
          "Yes, please, for tomorrow.",
          "Yes, Tomorrow morning please",

      ],
      "responses": [
        "Tomorrow morning is confirmed",
        "You can come tomorrow"
      ]
    },
        {
      "tag": "symptoms",
      "patterns": [
        "I am having Headache",
        "I have cough and nausea",
        "I have fever",
        "I am feeling weakness"
      ],
      "responses": [
        "Do you want to book an appointment?",
        "I prefer you to consult doctor. Do you want me to book an appointment",
        "Based on your symptoms you need to take covid test"
      ]
    },
  ]
}


In [67]:
# Preprocessing
# tokenizing

def tokenizer(sentence):
    doc = nlp(sentence)
    tokens = [token.lemma_ for token in doc if not token.is_punct]
    return tokens

In [68]:
def preprocess():

    all_words = []
    tags = []
    patterns = []

    for intent in chats['intents']:
        tag = intent['tag']
        tags.append(tag)
        for pattern in intent['patterns']:
            w = tokenizer(pattern)
            all_words.extend(w)
            patterns.append((w, tag))
    
    print('WORDS \n', all_words)
    print('TAGS \n', tags)
    print('PATTERNS \n', patterns)
    print(len(all_words))
    return patterns, all_words, tags

patterns, all_words, tags = preprocess()

WORDS 
 ['hi', 'hey', 'how', 'be', 'you', 'be', 'anyone', 'there', 'hello', 'good', 'day', 'good', 'evening', 'good', 'night', 'bye', 'see', 'you', 'later', 'goodbye', 'thank', 'thank', 'you', 'that', 'be', 'helpful', 'thank', 'be', 'a', 'lot', 'I', 'do', 'not', 'feel', 'well', 'I', 'feel', 'exhausted', 'I', 'be', 'tired', 'yes', 'please', 'for', 'tomorrow', 'yes', 'tomorrow', 'morning', 'please', 'I', 'be', 'have', 'Headache', 'I', 'have', 'cough', 'and', 'nausea', 'I', 'have', 'fever', 'I', 'be', 'feel', 'weakness']
TAGS 
 ['greeting', 'goodbye', 'thanks', 'Mood', 'Appointment', 'symptoms']
PATTERNS 
 [(['hi'], 'greeting'), (['hey'], 'greeting'), (['how', 'be', 'you'], 'greeting'), (['be', 'anyone', 'there'], 'greeting'), (['hello'], 'greeting'), (['good', 'day'], 'greeting'), (['good', 'evening'], 'greeting'), (['good', 'night'], 'greeting'), (['bye'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['thank'], 'thanks'), (['thank', 'you'], 'thanks'), (['

In [79]:
def bag_of_words(tokenized_sentence, words):
    """
    return bag of words array:
    1 for each known word that exists in the sentence, 0 otherwise
    example:
    sentence = ["hello", "how", "are", "you"]
    words = ["hi", "hello", "I", "you", "bye", "thank", "cool"]
    bog   = [  0 ,    1 ,    0 ,   1 ,    0 ,    0 ,      0]
    """
    sentence_words = [word for word in tokenized_sentence]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [80]:
# STEP 2: Define training data through a bag of words

X_train = []
y_train = []
for (pattern_sentence, tag) in patterns:
    x = bag_of_words(pattern_sentence, all_words)
    X_train.append(x)
    label = tags.index(tag)
    y_train.append(label)
print(all_words)
X_train = np.array(X_train)
y_train = np.array(y_train)

['hi', 'hey', 'how', 'be', 'you', 'be', 'anyone', 'there', 'hello', 'good', 'day', 'good', 'evening', 'good', 'night', 'bye', 'see', 'you', 'later', 'goodbye', 'thank', 'thank', 'you', 'that', 'be', 'helpful', 'thank', 'be', 'a', 'lot', 'I', 'do', 'not', 'feel', 'well', 'I', 'feel', 'exhausted', 'I', 'be', 'tired', 'yes', 'please', 'for', 'tomorrow', 'yes', 'tomorrow', 'morning', 'please', 'I', 'be', 'have', 'Headache', 'I', 'have', 'cough', 'and', 'nausea', 'I', 'have', 'fever', 'I', 'be', 'feel', 'weakness']


In [81]:
# dataset and dataloader

batch_size = 8

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

In [82]:
# model
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [83]:
# STEP 3: Configure the neural network

# define each parameter that is equal to 0 using an empirical value or a value based on your experience

num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)

# STEP 4: Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)

# Define loss and optimizer: which one is the best one?

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if(epoch+1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss:{loss.item():.4f}')

print(f'final loss: {loss.item():.4f}')


data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "hidden_size": hidden_size,
    "output_size": output_size,
    "all_words": all_words,
    "tags": tags
}

# STEP 5: Save the model
file = "med_assist.pth"
torch.save(data, file)
print(f'training complete. file saved!')

Epoch [100/1000], Loss:0.7485
Epoch [200/1000], Loss:0.1897
Epoch [300/1000], Loss:0.0569
Epoch [400/1000], Loss:0.0147
Epoch [500/1000], Loss:0.0075
Epoch [600/1000], Loss:0.0036
Epoch [700/1000], Loss:0.0037
Epoch [800/1000], Loss:0.0013
Epoch [900/1000], Loss:0.0014
Epoch [1000/1000], Loss:0.0013
final loss: 0.0013
training complete. file saved!


In [84]:
import random

In [85]:
# STEP 6: Test the model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load the intents file

# Retrieve the model and all the sizings
file = "med_assist.pth"
data = torch.load(file)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

# build the NN
model = NeuralNet(input_size, hidden_size, output_size).to(device)

model.load_state_dict(model_state)
model.eval()

# prepare a command-line conversation (don't forget something to make the user exit the script!)
bot_name = "Med Assistant"
print("Lets's have a chat with med assist! (type 'exit' to stop chatting!)")

while True:
    sentence = input("You: ")
    if sentence == "exit":
        print("Thanks for joining me!")
        break

    sentence = tokenizer(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X)

    output = model(X)
    _, predicted = torch.max(output, dim = 1)
    tag = tags[predicted.item()]
    probs = torch.softmax(output, dim= 1)
    prob = probs[0][predicted.item()]
    print(prob)
    if prob.item() > 0.75:
        for intent in chats['intents']:
            if tag == intent['tag']:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    
    else:
        print(f'{bot_name}: I do not understand. Try to be more specific')

Lets's have a chat with med assist! (type 'exit' to stop chatting!)
tensor(0.9979, grad_fn=<SelectBackward0>)
Med Assistant: What's up?
tensor(0.9994, grad_fn=<SelectBackward0>)
Med Assistant: Do you want to book an appointment?
tensor(0.7543, grad_fn=<SelectBackward0>)
Med Assistant: Bye! Come back again soon.
Thanks for joining me!
